In [0]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

SEED = 22
np.random.seed(SEED)

In [40]:
class SimpleGBRegressor:

    def __init__(self, n_estimators=10, max_depth=5, 
                 min_samples_leaf=1, learning_rate=0.1, 
                 random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.learning_rate = learning_rate
        self.random_state = random_state
        self.trees = []
        self.was_fit = False

    def fit(self, X, y):
        # убедиться что в X и y одинаковое число элементов
        assert X.shape[0] == y.shape[0]
        # не уверена, какой из вариантов лучше, поэтому вот второй:
        # assert len(X) == len(y)
        
        # инициализировать массив с предсказаниями, заполнив нулями
        prediction = np.zeros(y.shape)

        # обучаем деревья
        for i in range(self.n_estimators):
            # посчитать остатки
            residual = y - prediction

            # инициализировать дерево с нужными параметрами
            tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                         min_samples_leaf=self.min_samples_leaf,
                                         random_state=self.random_state)

            # обучить дерево
            tree.fit(X, residual)

            # сделать предсказание текущего дерева
            tree_prediction = tree.predict(X)

            # сохранить обученное дерево
            self.trees.append(tree)

            # обновить вектор предсказаний модели
            prediction += self.learning_rate * tree_prediction

        self.was_fit = True
        return self

    def predict(self, X):
        # если модель не была обучена, печатаем сообщение об этом и вовращаем None
        if self.was_fit == False:
            return None

        # инициализировать массив с предсказаниями
        y_pred = np.zeros(X.shape[0])

        # добавить прогнозы деревьев
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)

        # убедиться что в X и y одинаковое число элементов
        # снова не уверена, какой вариант лучше, поэтому:
        assert X.shape[0] == y_pred.shape[0]
        return y_pred


diabetes = load_diabetes()
data = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
target = diabetes.target
X_train, X_test, y_train, y_test = train_test_split(data,target,
                                                    test_size=0.3,
                                                    random_state=SEED)
regressor = SimpleGBRegressor(n_estimators = 100, random_state=SEED)
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)
mse = np.mean((y_test - y_pred) ** 2)
print('MSE: ',mse)

 

MSE:  3749.674167308474


In [41]:
from sklearn.ensemble import GradientBoostingRegressor

#сравниваем с реализацией аналогичного алгоритма
sklearn_model = GradientBoostingRegressor(learning_rate=0.1, 
                                          max_depth=5,
                                          min_samples_leaf=1,
                                          n_estimators = 100,
                                          random_state=SEED)
sklearn_model.fit(X_train,y_train)
y_pred_sklearn = sklearn_model.predict(X_test)
mse = np.mean((y_test - y_pred_sklearn)**2)
print(mse)

3725.914043073783
